Все импорты должны выполняться, особенно если вы запускаете это внутри докера

In [2]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import pandas.io.sql as psql

Вспомогательные функции для запроса данных из базы

In [4]:
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")

def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

def get_uid():
    return run_sql("SELECT uid FROM users WHERE data_loaded")

In [5]:
# Всего у нас вот столько таблиц

run_sql("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "13.84.188.132" and accepting
	TCP/IP connections on port 5432?


In [ ]:
# Посмотреть на все "колонки" в таблице можно вот так

run_sql("Select * FROM users LIMIT 0")

In [ ]:
# Так можно посмотреть на всех Ань

df = run_sql("SELECT uid FROM users WHERE first_name = 'Аня'")
df.head()

In [ ]:
# А так - сделать поиск по тексту комментария

df = run_sql("SELECT from_id, text FROM posts  WHERE (text <> '') AND (text LIKE '% да ладно тебе%')")
df.head()

In [ ]:
def getPostsByUid(uid):
    sqlQuery = "SELECT from_id, text FROM posts  WHERE (text <> '') AND (from_id = " + str(uid) + ")"
    return run_sql(sqlQuery).text

In [9]:
getPostsByUid(15759198)

0                                       хм...в смысле?)
1     не я все так удаляю))оставляю ток картинки и м...
2                    аххах ну это все для дебя детка)))
3     ой не повериш как я искала искала хд))а потом ...
4                           аххаха вот сейчас)))))хд)))
5                            пашка!!!а я тебя нашла))хд
6                                  урааа пасипки зай)))
7     хд да ладно тебе)))))))я же шучу насчет кулисо...
8     хд ну пашечка ну солнышко..ну малыш..ну кися))...
9                 аххах а ты меня возешь с собой???))))
10    пасип зайк))обязательно буду)))хахах)))*лыбить...
11    ооххохо прикольно))я та тоже не плохо это а ко...
12    оооо пашка ну ты отрастил патлы))))мужичек))))...
Name: text, dtype: object

In [10]:
# Допустим теперь для каждого юзера у нас появились предсказания, запишем их в сабмишн файл 

ids = get_uid()
leaders = np.random.choice(list(ids.index), size=100)


submission = pd.DataFrame({'is_leader':np.zeros(ids.shape[0])}, index=ids.index)
submission.loc[leaders] = 1

submission_file = '../data/submission/random_submission.csv'
if not os.path.exists(os.path.dirname(submission_file)):
    os.makedirs(os.path.dirname(submission_file))
submission.to_csv(submission_file)